In [7]:
from model.BSRModel import BSRModel


In [8]:
league =  'BZ'
season_start = 2016
season_end = 2064
pa_limit = 10
epochs = 8000
ratings_type = "Mixed"

In [9]:
bsrModel = BSRModel(league, season_start, season_end, ratings_type, "10")
bsrModel.load_data(pa_limit)

epoch_count, loss = bsrModel.train(epochs)
print(f"Epoch [{epoch_count}/{epochs}], Loss: {loss:.7f}")

test_loss = bsrModel.evaluate()
print(f"Test Error: {test_loss:.7f}")

#feature_importance = bsrModel.feature_importance()
#print(feature_importance)

bsrModel.save_model()

predictions = bsrModel.predict(season_end+1, pa_limit)
predictions.sort_values(by='Predictions', ascending=False, inplace=True)
print(predictions.head(10))

Empty DataFrame
Columns: [SPE, SR, STE, RUN, WT, BSR600]
Index: []
(22363, 6)
Epoch [2000/8000], Best Loss: 2.1749930
Epoch [4000/8000], Best Loss: 2.1504955
Epoch [6000/8000], Best Loss: 2.1388521
Epoch [8000/8000], Best Loss: 2.1305256
Epoch [7999/8000], Loss: 2.1305525
Test Error: 2.2718475
     SPE  SR  STE  RUN   WT  Predictions     ID
298   10  10   10   10  192     7.850183  60033
89    10  10   10   10  200     7.727069  53395
391   10  10   10   10  202     7.590488  62281
31     9  10   10   10  199     7.417643  50775
262   10  10   10   10  185     7.406497  59253
70     9  10   10   10  201     7.372244  52811
359    8  10   10   10  192     7.210056  61492
288    9  10   10   10  204     7.170693  59882
182    8  10   10   10  197     7.126129  57371
240    8  10   10   10  190     7.109239  58843
